In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras import datasets, layers, models
import tensorflow as tf

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import ReLU, concatenate
path='/content/drive/MyDrive/assignment 1/annotations.csv'
relative_path='/content/drive/MyDrive/assignment 1/train/train/'


In [2]:
def load_data(path,relative_path):
    df=pd.read_csv(path)
    data=[]
    for filename in df['fname']:
        address=relative_path+filename
        data.append(np.load(address))
    label=df['label']
    
    return data,label

In [3]:
data,label=load_data(path,relative_path)

In [4]:
mx_len=0
mn_len=100000
tot=0
for spectrograms in data:
    mx_len=max(mx_len,spectrograms.shape[2])
    mn_len=min(mn_len,spectrograms.shape[2])
    tot+=spectrograms.shape[2]
tot,mn_len,mx_len

(593504, 29, 2584)

In [5]:
def list_to_np(data,max_len):
    final_data=[]
    for spectrogram in data:
        x=spectrogram.flatten()
        y=np.zeros((max_len-spectrogram.shape[2])*64)
        y=np.append(y,x)
        z=np.zeros((max_len-spectrogram.shape[2])*64)
        y=np.append(y,z)
        final_data.append(y)
    return np.array(final_data)


In [6]:
def split_data(labels,data):
  split_data=[]
  split_labels=[]
  k=len(data[0])
  print(k)
  k=(int)(k/2)
  for i in range(len(data)):
    split_data.append(data[i][:k])
    split_data.append(data[i][k:])
    split_labels.append(labels[i])
    split_labels.append(labels[i])
  return np.array(split_data),np.array(split_labels)

In [7]:
data=list_to_np(data,mx_len)

In [8]:
data,label=split_data(label,data)

330752


In [9]:
def mfcc(data,mean=1):
  k=0
  if mean!=0:
    p=np.zeros((len(data),128))
  else:
    p=np.zeros((len(data),128,324))
  for i in range(len(data)):
    mfcc_feature=librosa.feature.mfcc(y=data[i],sr=44100,n_fft=2048,hop_length=512,n_mfcc=128,window='hann')  
    k=max(k,np.max(mfcc_feature))
    
    if mean!=0:
      mfcc_feature=np.mean(mfcc_feature,axis=1)    
      mfcc_feature=mfcc_feature.flatten()
      mfcc_feature=np.array(mfcc_feature)      
    p[i]=mfcc_feature
  p=p/k
  return p

In [12]:
label=pd.get_dummies(label)
mfcc_data=mfcc(data,0)

d=[]
for i in range(len(data)):
  k=mfcc_data[i].flatten()
  k=np.reshape(k,(324,32,4))
  
  d.append(k)
d=np.array(d)



In [13]:
train_x,test_x,train_y,test_y=train_test_split(d,label,test_size=0.2,random_state=9)

In [14]:
model = Sequential()
model.add(layers.Conv2D(128, (2, 2), activation='relu', input_shape=(324,32,4)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
history = model.fit(train_x, train_y, epochs=55)

Epoch 1/80
50/50 [==============================] - 36s 693ms/step - loss: 2.1992 - accuracy: 0.2100
Epoch 2/80
50/50 [==============================] - 33s 650ms/step - loss: 1.8490 - accuracy: 0.3613
Epoch 3/80
50/50 [==============================] - 33s 659ms/step - loss: 1.4470 - accuracy: 0.5244
Epoch 4/80
50/50 [==============================] - 33s 654ms/step - loss: 1.1532 - accuracy: 0.6119
Epoch 5/80
50/50 [==============================] - 33s 657ms/step - loss: 0.8525 - accuracy: 0.7206
Epoch 6/80
50/50 [==============================] - 33s 654ms/step - loss: 0.6729 - accuracy: 0.7800
Epoch 7/80
50/50 [==============================] - 33s 653ms/step - loss: 0.5276 - accuracy: 0.8188
Epoch 8/80
50/50 [==============================] - 34s 678ms/step - loss: 0.3747 - accuracy: 0.8781
Epoch 9/80
50/50 [==============================] - 33s 655ms/step - loss: 0.2880 - accuracy: 0.9050
Epoch 10/80
50/50 [==============================] - 33s 656ms/step - loss: 0.2057 - accura

KeyboardInterrupt: ignored

In [17]:
model.evaluate(test_x,test_y)

13/13 [==============================] - 2s 136ms/step - loss: 2.1725 - accuracy: 0.6950


[2.172529697418213, 0.6949999928474426]